<a href="https://colab.research.google.com/github/manushi0304/Diabetic_Retinopathy/blob/main/converted_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# --- Robust CM for TFLite FP16 DenseNet121 (tolerates wrong extensions / missing files) ---
import os, itertools, warnings
from pathlib import Path
import numpy as np, pandas as pd
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import tensorflow as tf
from tensorflow.keras.applications.densenet import preprocess_input as densenet_preprocess

warnings.filterwarnings("ignore")

# Reuse discovered paths from your previous cell:
# CSV_PATH = "drive/MyDrive/DiabeticProject/test.csv"
# TFLITE_PATH = "drive/MyDrive/DiabeticProject/tflite/DenseNet121_model_fp16.tflite"

IMG_SIZE = (224, 224)
NUM_THREADS = max(1, min(4, os.cpu_count() or 1))
TRY_EXTS = [".jpg", ".jpeg", ".png", ".bmp", ".tif", ".tiff"]
SKIP_MISSING = True  # set False to raise if any image is not found

def _coerce_label(x):
    if isinstance(x, (np.integer, int)): return int(x)
    if isinstance(x, str):
        s = x.strip().lower()
        mapping = {"no_dr":0,"no dr":0,"normal":0,"0":0,"dr":1,"referable_dr":1,"referable dr":1,"1":1}
        if s in mapping: return mapping[s]
        try: return int(float(s))
        except: pass
    raise ValueError(f"Cannot coerce label '{x}' to int 0/1.")

def _fix_path_if_missing(p):
    """If path p doesn't exist, try alternate extensions and a fuzzy stem search."""
    if os.path.exists(p):
        return p
    base, ext = os.path.splitext(p)
    # Try swapping extension
    for e in TRY_EXTS:
        q = base + e
        if os.path.exists(q):
            return q
    # Fuzzy search by stem in the same directory
    parent = Path(p).parent
    stem = Path(p).stem
    if parent.exists():
        cands = list(parent.glob(stem + "*"))
        # prefer image files
        cands = [str(c) for c in cands if c.suffix.lower() in TRY_EXTS]
        if cands:
            return cands[0]
    return None

def load_csv_and_paths(csv_path):
    df = pd.read_csv(csv_path)
    cols = {c.lower(): c for c in df.columns}
    label_col = next((cols[c] for c in ["label","labels","diagnosis","target","y"] if c in cols), None)
    if label_col is None:
        raise ValueError(f"No label column found in {list(df.columns)}")

    path_col = cols.get("filepath", cols.get("path"))
    id_col = cols.get("id_code", cols.get("image", cols.get("id")))

    if path_col is None and id_col is None:
        raise ValueError("CSV must have either a filepath/path column or an id_code/image/id column.")

    img_paths, labels = [], []
    missing = 0

    if path_col is not None:
        for _, r in df.iterrows():
            p = str(r[path_col])
            if not os.path.isabs(p):
                p = str((Path(csv_path).parent / p).resolve())
            q = _fix_path_if_missing(p)
            if q is None:
                missing += 1
                if not SKIP_MISSING:
                    raise FileNotFoundError(f"Image not found: {p}")
                continue
            img_paths.append(q)
            labels.append(_coerce_label(r[label_col]))
    else:
        # id_code mode -> try common folders near CSV
        csv_dir = Path(csv_path).parent
        cand_dirs = [
            csv_dir/"images"/"test", csv_dir/"images"/"val", csv_dir/"images",
            csv_dir/"data"/"images"/"test", csv_dir/"data"/"test", csv_dir/"data"
        ]
        cand_dirs = [d for d in cand_dirs if d.exists()]
        if not cand_dirs:
            raise FileNotFoundError("Could not infer images directory for id_code; create one near the CSV.")
        for _, r in df.iterrows():
            pid = str(r[id_col]).strip()
            q = None
            for d in cand_dirs:
                for e in TRY_EXTS:
                    t = d / f"{pid}{e}"
                    if t.exists():
                        q = str(t); break
                if q: break
            if q is None:
                missing += 1
                if not SKIP_MISSING:
                    raise FileNotFoundError(f"No image for id '{pid}' in {cand_dirs}")
                continue
            img_paths.append(q)
            labels.append(_coerce_label(r[label_col]))

    if missing:
        print(f"[Warning] Skipped {missing} rows with missing images; proceeding with {len(img_paths)} samples.")
    return img_paths, labels

def load_and_preprocess_image(path):
    img = Image.open(path).convert("RGB").resize(IMG_SIZE, Image.BILINEAR)
    arr = np.asarray(img, dtype=np.float32)
    arr = densenet_preprocess(arr)
    return np.expand_dims(arr, 0)

class TFLiteModel:
    def __init__(self, model_path, threads=NUM_THREADS):
        self.interp = tf.lite.Interpreter(model_path=model_path, num_threads=threads)
        self.interp.allocate_tensors()
        self.inp = self.interp.get_input_details()[0]['index']
        self.out = self.interp.get_output_details()[0]['index']
    def predict_one(self, x):
        self.interp.set_tensor(self.inp, x)
        self.interp.invoke()
        return self.interp.get_tensor(self.out)

def decide_binary_preds(outputs):
    outputs = np.asarray(outputs)
    if outputs.ndim == 1: outputs = outputs.reshape(-1, 1)
    if outputs.shape[1] == 1:
        probs1 = 1/(1+np.exp(-outputs.squeeze()))
        preds = (probs1 >= 0.5).astype(int)
        return preds, probs1
    elif outputs.shape[1] == 2:
        logits = outputs - outputs.max(1, keepdims=True)
        probs = np.exp(logits) / np.exp(logits).sum(1, keepdims=True)
        return probs.argmax(1), probs[:,1]
    else:
        raise ValueError(f"Unexpected output shape {outputs.shape}")

def plot_confusion_matrix(cm, classes=(0,1), title="Confusion Matrix", save_path="confusion_matrix_tflite.png"):
    plt.figure(figsize=(5, 4.5), dpi=120)
    plt.imshow(cm, interpolation="nearest", cmap=plt.cm.Blues)
    plt.title(title); plt.colorbar()
    ticks = np.arange(len(classes))
    plt.xticks(ticks, classes); plt.yticks(ticks, classes)
    plt.xlabel("Predicted label"); plt.ylabel("True label")
    thresh = cm.max()/2 if cm.max()>0 else 0.5
    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, f"{cm[i,j]:d}", ha="center",
                     color="white" if cm[i,j] > thresh else "black",
                     fontsize=12, fontweight="bold")
    plt.tight_layout(); plt.savefig(save_path, bbox_inches="tight"); plt.close()
    print(f"[Saved] {save_path}")

# ---- Run ----
print("Loading CSV and resolving image paths...")
img_paths, y_true = load_csv_and_paths(CSV_PATH)
y_true = np.array(y_true, dtype=int)
print(f"Using {len(img_paths)} images for evaluation.")

print("Loading TFLite model...")
model = TFLITE_PATH
interp = TFLiteModel(model)

print("Running inference...")
outs = []
for i, p in enumerate(img_paths, 1):
    outs.append(interp.predict_one(load_and_preprocess_image(p)).squeeze())
    if i % 100 == 0 or i == len(img_paths):
        print(f"Inferred {i}/{len(img_paths)}")

y_pred, _ = decide_binary_preds(np.array(outs))

cm = confusion_matrix(y_true, y_pred, labels=[0,1])
acc = accuracy_score(y_true, y_pred)
prec = precision_score(y_true, y_pred, zero_division=0)
rec = recall_score(y_true, y_pred, zero_division=0)
f1  = f1_score(y_true, y_pred, zero_division=0)
print(f"Accuracy:  {acc:.4f}\nPrecision: {prec:.4f}\nRecall:    {rec:.4f}\nF1-score:  {f1:.4f}")

plot_confusion_matrix(cm, classes=(0,1), title="Confusion Matrix", save_path="confusion_matrix_tflite.png")


Loading CSV and resolving image paths...
[Warning] Skipped 3662 rows with missing images; proceeding with 0 samples.
Using 0 images for evaluation.
Loading TFLite model...
Running inference...
Accuracy:  nan
Precision: 0.0000
Recall:    0.0000
F1-score:  0.0000
[Saved] confusion_matrix_tflite.png
